In [ ]:
# create an empty Df with the fighter names as the col headings.
fan_detect = pd.DataFrame(columns=all_fighter_cols)

#find the proportion of comments a user makes about specific fighters N.B each user will have a column for every fighter
for column in all_fighter_cols:
    
    fan_detect[column] = fighter_measure[column]/fighter_measure["total_comments"]

fan_detect["total_comments"]= fighter_measure["total_comments"]

user_array= []
names_array = []
values_array = []
totals_array = []

#iterate through rows, and find the fighter with the highest proportion of comments for each user
for _, row in fan_detect.iterrows():
    
    #locate column heading with highest proportion fighter
    max_name = row[all_fighter_cols].idxmax()
    # take value of cell at that column
    max_value = row[max_name]
    
    # discard users who do not mention any fighters
    if max_value != 0:
        #collect usernames
        print
        user_array.append(row.name)
        # collect each users most popular fighter
        names_array.append(row[all_fighter_cols].idxmax())
        # collect value for most popular fighter
        values_array.append(max_value)
        # collect non discarded users total comments
        totals_array.append(row["total_comments"])



average_array = []
key_array= []
# find the expected highest proportion of fighter mentions wrt total user comments
# groupby the total comments, and average results,giving the expected numebr of mentions for a given number of comments.
for key, sub_frame in fan_detect.groupby(["total_comments"]):
   
    values_array1= []
    for _, row in sub_frame.iterrows():

        # for each row (now indexed by total comments) find the col and value of the max proportion fighter
        max_fighter = row[all_fighter_cols].idxmax()
        value_max = row[max_fighter]
    
        if value_max > 0:
        
            values_array1.append(value_max)
    #collapse sub frames (grouby object grouped by number of comments) into into a mean
    average_array.append(np.mean(values_array1))
    key_array.append(key)
    
#find the average rate of favourite fighter mentions for a given number of comments, rolling mean smooths the line
mov_ave_array = pd.rolling_mean(pd.Series(average_array),40)


#create line that indicates the expected rate of favourite fighter mentions
fanatic_line = pd.DataFrame({'total comments': key_array, 'expected score': mov_ave_array}).sort_values('total comments', ascending=False)

#data of all users to plot
all_fans = pd.DataFrame({'user': user_array,'fav fighter': names_array, 'fav proportion': values_array, 
                         'total comments':totals_array })
all_fans = all_fans.sort_values(['total comments'], ascending=False)


# determine which fans mention fighters more than expected
fanatic_fans = all_fans.copy()
fanatic_fans = fanatic_fans.merge(fanatic_line[['total comments','expected score']], on=['total comments'])

#determine the rate above the mean that is considered 'unusual' 
fanatic_fans['cut off score'] = fanatic_fans["expected score"] * 1.5  
fanatic_fans = fanatic_fans[fanatic_fans["fav proportion"]>fanatic_fans["cut off score"]]
fanatic_fans = fanatic_fans[fanatic_fans["total comments"]> 0]
fanatic_fans = fanatic_fans.set_index(fanatic_fans['user']).sort_values('fav proportion', ascending=False)
fanatic_fans = fanatic_fans.drop('user', 1)